In [32]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import json

In [33]:
from konlpy.tag import Okt
okt = Okt()

In [34]:
openFileName = '전자담배 금연_naver_news_1000.json'
    
f = open(openFileName, 'r', encoding='utf-8').read()

jsonData = json.loads(f)
data = ''

for item in jsonData:
    if 'description' in item.keys():
        data = data + re.sub(r'[^\w]', ' ', item['description']) + ' '
        
# f.close()

In [35]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [36]:
import gensim

stop_list=['꼭좀', '부여', '체워', '하나', '무작정', '매탄', '뭔들', '연천군', '걸사', '피셧는데', '하제', '바로가기', '대체', '되셧기', '불도', '정진욱', '핑거', '아무', '객실', '거치', '개월내', '전라남도교육청', '하루아침', '지랄', '트리', '갖가지', '답좀', '드하', '나위', '글로', '세지', '홀로', '릿는데', '최소', '극렬', '해면', '생기지', '이다해', '폐홉용', '여섯', '래시', '구지', '반듯이', '차라리', ' 두번재', '다가', '필욘', '데용', '솜에젹셔진', '악기', '가름', '커뮤', '박영선', '정해진', '만원', '만만만', '최종', '이면', '간다', '경기', '용산동', '중산동', '에나', '답글', '능돠', '일산', '깜박', ' 만만', '지마', '크게', '곤찮', '프레', '동인', '갯수', '두둥', '테워', '루크', '에소', '구로', '셨습', '포항시', '이씨', '저절로', '뜻임', '한해', '까르', '구우므', '부들부들', '나볼', '쑤닝', '최소한', '드믄', '글치', '윈윈', '여러분', '답글달', '봉명동', '서하', '허투루', '성주', '여타', '글썽거렷', '아무나', '현격', '맘속', '걸껀디', '방선휘', '더움', '지구', '답주', '김이랑', '리버', '기스나', '이상은', '대명사', '참고문헌', '다솜', '가지', '질끈', '최씨', '훨씩', '현저', '막말', '작년', '가닥', '일어나고', '꼿는', '목구', '꼬득', '세터', '가각', '난뒤', '헤휴', '부시', '굳이', '가디언', '김경옥', '운옹', '딸도', '다섯', '질물', '착착', '지난번', '제등', '향나', '구렇다', '것입', '나기', '수백', '나가야', '관련', '부산광역시', '궐리티', '타시기', '지란', '레씨', '뽐핑', '가마', '덮어놓고', '더욱더', '서울', '삼은새벽', '쫩쫩', '뻥뻥', '나날이', '분명', '셧음', '잠깐', '부천시', '매번', '사가정역', '호완되', '바로', '무티', '안녕', '각국', '통째', '귾는', '허세쩌', '위키', '거닌', '비슷', '구경해', '멋브럽다', '촛점두어', '해피빈', '떄문이', '대신', '마포', '진정', '느겨', '이수', '수천', '토머스', '댕겨', '노형동', '관해', '자첵하', '딴말', '광주', '인지', '알곗더', '쥐도', '르그', '뻘쭘', '네나', '래지', '못바꿧어', '강원', '점철', '레습니', '도중', '브라', '리빌드아님', '견재하', '달치', '드릴', '내외', ' 제만', '피는량', '부산', '김태원', '라나', '하실껀', '들어왓다', '대체재', '내시', '답변', '댈껍니', '비록', '로크', '살띠', '고려대', '넘김', '어딘로', '차근차근', '딲기', '멈칫', '흉내내', '문하', '묵힐', '개연', '막굴', '오산', '연계퀵', '캐시', '남동구', '이전', '해성', '타고', '님께서', '일찍', '어기', '단자', '요전', '치인', '충청남도', '즉시', '쪄듭니', '헹굴', '언젼히', '메모', '달마', '철갑상어', '죽겠네', '즉슨', '고타르', '돈까스', '우쩨뜬', '유튭', '김순', '어불', '단테', '얼떨결', '도농', '산이면', '부차', '붙이', '대다수', '기존', '눈앞', '년월', '단계', '헬조선', '구찮으', '멀쭘하', '소공로', '겨우', '투잡', '깜빡', '안해', '란데', '개월때', ' 공주대학교', '로사', '번의', '아래', '팔리니', '불보듯', '님디', '괜춘함', '홧팅', '필때나', '대연동', '덕이', '안양', '심혁관', '훠월씬', '장재', '한다해', '끝일', '데이비드', '절로', '요게', '나마', '격한거', '담해', '민하', '찟어질', '통밀', '랄것', '뭣모르', '때시', '단지', '뭣좀', '쪼끔', '매기', '네프', '롯도', '담시담좀', '한창', '모자라', '교유', '도실', '최적', '복정동', '어떠케', '버금', '하버드대', '용산구', '량이므', '흡힙해', '다뻬먹는', '빤스', '삼고', '라미', '갑반', '괜찬은것', '충남', '파서', '뻑뻑', ' 길어졋내', '쭈러쭈러', '획일', '늘더', '증말', '랍니', '한세대', '무량', '어도', '그녀', '잠정', '그롬', '뉴스투데이', '인제', '도도', '오늘', '조금', '한님', '거구', '간지', '나이라', '뭐임', '잔뜩', '마틴', '우뚝', '금정', '김정은', '다드', '일이', '도대체', '는바', '내심', '괜찬나', '정국', '두루', '달렷답', '한번', '딕', '대저', '더미', '자임', '물로씼거', '대재', '허지만', '봉명', '소릴', '해운대', '매일', '질때', '무등', '은지', '류량', '걸어봐욤', '흐트러진', '대가', '디즈니', '당근', '거려', '써브옴', '완제', '갈마동', '감사', '고양이', '녹슨', '퓻퓻', '서도', '괴면적', '자꾸', '다시마', '하셧을테구', '일본여행', '최적화', '로되', '그죠잉', '가정동', '다아나', '무심코', '감초', '쟁졈', '기타등등', '동남구', '통큰', '은데', '괸찮', '고치시', '비숫', '흡사', '될런', '안현모', '동일', '대구', '두운', '어쨌든', '가유', '개교', '차하', '눈팅', '저번', '김포', '초록색', '님코', '랜드마크', '도나', '몰래', '최초', '마곡나루역', '듭니', '여의도역', '쌀짝', '천국', '팁좀', '아실껍니', '순례', '대니', '왼쪽', '노래', '준수', '휠씬', '파이팅', '제때', '구월동', '돌리시', '끈끈', '구로구', '가경동', '특정', '불구', '남부', '입잼새', '눈차', '날쭉', '브리', '만해', '늘격는', '이글', '김성모', '살레', '리브', '나왓다구', '바트', '마구잡이', '지게', '단번', '만만', '가축', '지엇습니', '드렷듯', '꼇다', '함등', '범용', '청미래덩굴', '뭐시기', '님과같', '있엇어', '슈류탄', '리움', '회원구', '라발', '한지', '제목', '님께', '매도', '누스등', '달남짓', '대하', '기성은', '러시', '다압니', '박진주', '휜것', '네비게이션', '퍼프바', '내내', '만종', '주말', '느겼을뿐', '것담', '구기', '요컨대', '부천', '멀티샾', '거립니', '일시', '바람', '다나', '첫날', '취한', '이내', '훌룽햐', '요것', '체텍', '가정', '줄줄', '경양', '가나', '썻다', '충남대학교', '밍밍', '겟습', '거땜', '관우회', '제곧내', '아야', '법등', '기겄해', '막무가내', '얼척', '일명', '라신', '디자이어', '오시', '레이', '하겟습니', '항용', '더군다나', '사각', '언제', '퍼뜨리세', '유성', '무미', '처럼', '입홉', '이어도', '금엱', '자꾸만', '각심', '시마', '되억네', '로키', '레소', '닉액', '계제', '흥덕', '화성', '한느데', '괸히', '히스토리', '심심찮', '워낙', '한경닷컴', '으쓱', '매가', '다해', '개수', '됫는데', '취할', '가방', '서면', '모름', '뗘내', '포항', '잘못', '건아', '것도', '데이', '남짓', '대고', '도움말', '말로', '할껀데', '데렉', '대략', '길음동', '개인', '주눅', '헛것입니', '성심껏', '이영자', '두서', '건양대', '몇개', '파이뽀', '식지', '주드', '끼니', '능사', '누리', '하나로', '충북', '얼렁', '믹트', '탑깝습', '가도', '피시', '추어', '다오', '권해듸', '답글답니', '두번째', '다독', '다릉것', '괜시리', '직딩입니', '성북', '사실', '구성은', '브이', '박천', '탈것', '바라지', '음으로', '프린터', '즉그', '동아제약', '사이드', '영웅문', '카카오', '거듭', '탈때', '슝슝', '개사', '달새', '링크', '맛맛', '안눌', '모로', '과다르', '흐허', '세라', '꼭두각시', '나나', '더구나', '꺽쇠', '시샤', '용인시', '대전', '깜깜', '만끽', '해봣는데', '상황', '우수수', '금염', '다재', '알ㄹ리', '잠도', '고려', '눌려', '페이지', '직전', '당황', '번디', '과언', '릴사려', '소이', '괸찮습니', '커니', '답타', '말슴들', '타네', '그다지', '단도', '봣을때', '클릭', '듯이', '주구', '빈틈', '오늘이', '한층', '분당구', '래미안', '프랑', '오래오래', '를피', '넥타이', '업청', '광주시', '얼핏', '다도', '도체', '오신', '관악', '방시으', '바꼇어', '이군', '졸지', '인젠', '구울', '최우수', '가인', '드타', '실질', '만큼', '동안', '데이타', '곧곧', '햣으', '뿜뿜', '몬스처', '썻느데', '곳도', '중랑', '무렵', '일욜날', '극히', '스모', '박종화', '어쩧', '오링', '낳을', '바서', '마는', '나머지', '얼른', '비릿하', '하물며', '시긴', '필떄', '으리', '부동산', '을지', '절대', '김승희', '최혁', '저희', '일괄', '면액', '별별', '까치', '경남', '이구', '건건', '강도', '니드', '릴미', '때기', '사리', '기간', '봐놧는데', '팟형', '동동', '사서', '조윤정', '메시', '자양동', '멋대로', '반짝', '곤충', '홈페이지', '입폐', '혼자', '물일', '납득', '리빌드형', '조선일보', '낱낱이', '랫습', '궁굼한거', '어디가', '감히', '가늠', '정해', '가뜩이나', '네임드', '삼만원', '여수', '가수', '릴플', '전후', '호읍', '게시판', '도로', '하겟좋', '헥헥', '톨레도', '고로', '해도', '짐작', '게뭐', '통령', '만오천원', '문장', '중간', '등지', '궁비', '그날', '하든', '엇죠', '별내동', '겅분', '괜찬다', '누가', '조굼싹', '저쩌구', '꼬나물', '못햇습니', '팩트', '가까이', '플린트', '이듭', '또말', '헤럴드', '르레', '면솜', '헹궈주', '디저트', '강북', '린치', '랙트', '건가', ' 뱅이', '법좀', '그땐', '도졋네', '더니', '그렇케', '쭉쭉', '뉴스레터', '쓱쓱', '작뜩', '잘만', '어카', '최신', '향향', '자도', '호기', '되엇습니', '그저께', '달짜', '노릇', '프리', '차단하라', '현충로', '컵라면', '압구정', '말함', '야마하', '더욱이', '뭔가', '피셔', '일개', '마르고', '지금껏', '금도', '관리겟', '광진', '대구광역시', '고자', '다한', '최후', '남눈', '서울특별시', '어쩌구', '도달', '거란', '하셧는대', '동해', '괴정', '다시', '할껀', '닌깐', '서든', '가로막', '끽연', '차후', '일쑤', '어서', '사도', '구나', '역곡역', '노원구', '이하', '장보고', '깝습', '마눌', '마이크', '갑사', '사나', '로간', '묶인', '둘째', '겻던', '여주시', '해주', '함안군', '뉴비', '봉구', '무대', '무었', '개월치', '주머니', '물성', '거싱', '맘껏', '인걸', '자체', '한쎄트', '껍니', ' 두고두고', '천남', '몇몇', '다시금', '꼬르륵', '도민', '율량동', '각자', '남고', '고타', '첫번쨰', '망우동', '만천하', '믈질', '구경', '어데', '꿍짝', '셧다', '처음처럼', '뭉텅', '말리', '관리', '가요', '과연', '김철중', '네오', '이르키', '얼머', '눈시울', '대화기', '릴같', '빠듯', '동해시', '내후년', '하실껍니', '더라도', '하나라', '키고눌럿더', '맛나', '더원', '마음껏', '하셧으', '컨데', '한일', '아래쪽', '제라', '꼭추천즘', '후루룩', '개피', '오니', '동네', '경북', '자이시', '차차', '마누라', '성설', '에다가', '궁굼한', '팔지', '김명환', '문제기', '년호', '마넌', '면암', '완다', '내지', '박경환', '무조건', '헬스장', '드니', '스샷', '재는', '여지', '대만', '헤롭나', '페이스북', '핏다호환되', '톡톡', '뎅직', '블라블라', '랜들', '바이', '무리수', '중요시', '공룡', '쥐꼬리', '일일이', '도고', '때입', '고차', '김진영', '여야', '었어', '엠씨', '하나요', '중이', '김동주', '도우', '진젇', '삼은', '러하', '팔면', '린지', '암만', '적색', '가게', '쏠쏠', '팔만', '벌써', '그게', '못합', '이조이', '이서', '말햇', '원평동', '스케줄', '두두', '셨듯', '회지', '고향', '로써', '통관', '리큐', '썻는데', '블루', '파리', '저런', '골프', '고스', '살임', '드믐니', '달후', '숨기', '갑자기', '마더', '줏으러', '부문', '금번', '동일로', '오른', '괜찬', '모토로', '불완전', '디자이너', '금정구', '내년', '눈알', '강동구', '꾸욱', '더하기', '넣쿠', '로햇', '뭉치', '부교수', '어던', '쭈욱', '누나', '은제', '뚝섬', '서기대', '지그', '취해', '궁굼해', '오히려', '살짝', '흥이', '몬가', '목하', '부산시', '치전', '똑바로', '챙피', '엇답', '이단', '꼽아', '어쩌고해', '쌘건가', '우리', '치기', '란것', '더몸', '건데', '되셧으면그', '산지', '개개', '볼때', '달도', '보닐류등', '피얼때', '전은영', '구서', '가주', '저걸', '심심', '햇습', '파도처럼', '따로따로', '뮤슨', '삼이', '흐트러져잇음', '털털', '해선', ' 청원구', '마안', '고이', '햇갈렷습니', '초록불', '지난주', '뿐더러', '기네', '죠즈', '돈대', '한참', '날때', '무척', '디프입니', '들보', '서귀포', '열휼값', '찻순갈', '모도', '미국인', '당성', '시청', '아작', '당췌', '당한', '간나', '문건', '기릴', '노락색', '넷째', '중국인', '에고', '되셧네', '강남역', '코닌', '불순', '그었다', '어덯', '함부로', '경주', '넘버', '찬티론', '해운대구', '근간', '놨는데', '달함', '마인', '시청역', '명보', '까페', '감자', '하셧는', '뜬구름', '원성동', '흥덕구', '자양', '맘니', '칭다오', '주지', '피어', '있을껍니', '이천시', '괜찬은', '양으로', '밥버거', '미소', '부로', '가부', '부분', '태운', '대엽', '작성', '스위', '님키', '팟들', '사상구', '마침내', '떄문', '옴일', ' 지난', '란느', '목도', '서요', '랄임', '핑크', '젭알', '완후이', '섣불리', '하루에하나씩', '미주', '사뭇', '파라', '천년', '선봬', '개코', '메스머', '골문', '논현동', '크리', '되갯', '강서구', '구월', '높힙니', '원리', '소코', '맘에들면', '이상일', '쪼금씩', '타요', '남천역', '쇼미더머니', '기계로', '진웅', '꿈틀', '헥헤', '라르', '재미나', '개입', '다기', '쭐려가면', '롯켓', '게인', '앞서서', '곽을보', '비씬거', '나오니', '반대론', '다니엘', '임도', '까운', '왜냐면', '삼겹살', '로그', '기쁨', '무화량얇', '그럿', '제겐', '성신여대', '끝으셔', '목화', '도해', '발품팔', '휙휙', '맺힌', '리마', '개월됫는데', '만듭니', '물임', '폐홉', '어마', '헐엇어', '팀등', '끌끌', '번쨰', '뭍혀', '타운', '어쨌다', '마침', '뿌리', '홍정표', '저건', '나베', '님들', '는고', '왓답', '도씨', '도배', '이어진', '살기', '강영호', '드렇', '면서', '서현역', '간장', '십분', '쒸우세', '년금', '아뇨', '시오', '답글답', '냄비', '연간', '평상시', '미라', '이처럼', '며더', '훗근데', '회도', '사람인', '답시고', '편더', '냇째', '담양군', '파란색', '해보렵니', '햇구', '쌍발', '민국', '지안', '별개', '후자', '구해', '위해', '량에따', '다뺸건데', '어차피', '보시', '죽빵', '세번', '남산동', '눌린', '카르', '널리', '대면', '관찮나', '대부분', '강사', '일어', '서두', '세팅할', '상떼', '따라서', '린다', '다해봣', '인현동', '다소', '점점', '살갖', '머릿속', '만듬새', '김동석', '돌잔치', '드뭅니', '삼성동', '살살', '미명', '구미', '대로', '러듭', '가파르', '도오', '싸구려', '이크', '이지은', '님한테', '리프', '나름', '몹시', '빵빵', '또한', '양구군', '대폭', '서장훈', '퉤퉤', '해쥬', '맞이', '달뒤', '나니', '당시', '개월됫구', '팔고', '헀습니', '가드', '대리점', '집밖', '갭슐', '제법', '덕지덕지', '좋곘습니', '다자', '파니', '폇는데', '드렷', '만일', '강남구', '김일순', '생뚱맞', '갈채', '천배', '뿌르르르', '종은', '이홍', '마르지', '리더', '연무', '동창회', '거만', '달달', '우편번호', '친철', '버젖', '종종', '상태', '뭐내', '경상북도', '상봉동', '답답증', '껌등', '가능도', '백배', '계양', '여서', '라시', '더혜', '니코니코', '도어', '갖은', '사가', '더피', '고극', '델겁니', '역대', '색도', '써비스', '이제일', '중국사', '강츄입니', '다다', '도란', '때매', '사이사이', '바짝', '요음', '남지', '글구', '영등포', '필요시', '뜬금', '부활동', '립니', '도구로', '밀면', '이틀', '땡큐', '좀더', '짜르', '달서구', '텄다', '김태', '백만배', '망간', '닉네임', '유디', '보람', '어이', '킷입', '금방', '바탕화면', '물자', '학업떄문', '소위', '부천우편집중국', '원미구', '눈초리', '럿다', '찾기', '당신', '더괞찮으', '기필코', '달라', '김치', '가스나', '부산지하철', '개월', '해볼껀', '워런', '길음', '데액', '거릴거', '항상', '모금', '이성규', '중랑역', '어보', '길눈', '이외', '보기보더', '쭈욱가세', '조인성', '초희', '번만', '됫어', '뽕삥', '두발', '니닐', '박인숙', '댓으', '됫네', '일이만원', '같구욤', '하내', '두산백', '퍼온겁니', '쪽말', '못봣', '문턱', '벌컼벌컼', '구간', '독일', '대도시', '한국인', '말경', '요시', '클리', '조성흠', '그떄', '얼맙', '게타', '경염', '마찬가지', '달쫌', '한단', '어쩜', '것일', '중인', '괜춘', '안양시', '토햇습니', '위챗', '껄유', '이제야', '펴왓으', '파가', '로부터', '요즈', '드랴', '메뉴', '날자', '충무로', '디지털로', '도무지', '차라', '마이', '뻥튀기', '정히', '비지땀', '젤류', '백일', '요즈음', '려면', '용인대', '망우역', '날수', '꼬박꼬박', '취할려', '어쩌면', '영등포구', '이곡역', '폐쇄되어', '디테', '카드', '꼬우해', '될껀데', '항료', '인천', '쿠쿠', ' 훗날', '비비용', '복은', '서서', '못내', '순서대로', '돌이', '드립', '수준', '릴핏', '모든', '젓갈', '미만', '사면', '창원', '방금', '쭐여', '됫습', '언뜻', '그다음', '율량쪽', '대해', '리오', '비롯', '짝통', '통과', '게다', '차만', '물씬', '밀리', '나쓰나', '겨냥', '단독주택', '댓글', '래그', '얻지', '보텅', '서나', '진짜', '쇠사슬', '막상', '경산', '경상남도', '계양구', '글쎄요', '폇거든', '대동', '뭐드', '뿜어', '거림', '지금', '내재', '솜이므', '파고', '훌쩍', '부평', '버니', '쪼금쌔', '각의', '프린스', '문의사항', '모되', '팔콘', '메이비', '시엔', '뎅저', '젹셔지', '졋던', '유라', '점좀폐', '아타', '종로구', '연한', '치마', '천곡', '오거', '라그', '난후', '엮겁', '유성구', '님드', '트려', '햇데', '모습', '중해', '박해', '매몰', '타라', '반면', '남쪽', '보이', '스컬', '어쩐지', '통로', '신동엽', '문정역', '력도', '김씨', '끝내', '안녕하십니까', '설령', '부방', '즈음', '동아일보', '어아', '억지로', '무지막지', '맛강', '피속', '구경오', '감격', '거세', '단디', '최대', '니코', '안녕하세요우', '맏았', '젤괸찬은거', '더블', '소파', '위함', '요새', '있길레', '개강', '자구', '다담', '송정동', '다운', '개재', '마셈몸', '물금', '때뇌', '도림동', '깜놀', '백만', '번답', '무어', '앤더슨', '나목', '이후', '티티', '원래', '만날', '주간', '다수', '야호', '장조림', '계세', ' 밤새', '김모', '나은지', '력기', '기회', '면알', '아주', '답터', '대추', '지니', '님게서', '진진', '가후', '싸이트', '후밴님들', '맑은', '때세', '은근', '바슷', '라두스', '기어', '바꿀껀데', '간의', '먹기', '가제', '튀김', '사셧', '나용', '답배', '겐찮', '더반', '고라', '어요', '새록새록', '단국대', '화이팅', '강북구', '만이', '무주', '액삭', '게꼭', '앰팽', '저쩌', '액속', '넷상', '통틀어', ' 두루', '깜짝', '나진', '한눈', '헷었', '강하니', '뽐쁘하', '찿는', '햇스', '드나', '쌍수', '사라', '종류', '매달', '에손', '실컷', '동쪽', '무조', '툭툭', '걸렷거든', '한경', '확늘엇습니', '이동수', '리차드', '체우', '세고', '팔이', '거에욤', '컷다', '중랑구', '두께', '무너', '계이름', '때문', '명구', '대한', '거도', '워드', '퍼니셔', '왕창', '기니', '중순', '제대로', '삼시', '대공원역', '자칭', '견우', '과정', '목화씨', '빡빡', '달동', '늑대', '깨꿋하', '듯함', '깊이', '대사', '이참', '못햇', '사이', '야기', '무지', '지나', '죽죽', '깨소금', '한텐', '빽뺵', '산괴', '전주시', '평소', '경우', '보헴', '해주시', '전라남도', '너프', '자칫하면', '구원진', '송중', '고추', '다만', '정영기', '쌘건', '용인', '관양동', '가야', '다른', '로서', '몇장', '주기', '즉각', '박능후', '덕점', '테엽같', '마리', '무사', '우후죽순', '삼성', '녀석', '쥴같', '걸론', '박사', '듣기', '해치', '부처', '앙드레', '동시', '대내', '계산동', '역곡', '개인젇', '노원역', '주니', '적접', '풀때', '말착', '윤정훈', '느낌', '듭니다많', '끈끈이', '뉴스', '강정구', '대충', '주립대', '가심', '달차', '답답', '미력', '와스', '로스', '바보', '최근', '쓰시', '바꿧', '게다가', '통등', '라며', '라라', '빨렸는데', '누더', '개외', '따님', '종일', '니니', '웠분말', '대요', '정일우', '발라', '다됫', '개미', '구조만', '사거리', '더욱', '톡방', '류액', '내공', '프로필사진', '그것', '비로소', '라가', '극력', '차츰차츰', '눈물', '경향', '어닝', '펌핑하', '장연', '나야', '만남', '꼬락서니', '여기저기', '별도', '줄때', '로치', '조목조목', '작심삼일', '계피', '레시', '폇는데요엠창', '삷에대', '종이', '뜨껍', '로안', '압니', '간만', '고하', '지난달', '마름', '줕혀놓기하', '차등', '가왜', '배등', '송파', '그거떄문', '할껍니', '트린', '전라북도', '마냑', '한마음', '명일', '별것', '거나', '몇번', '뎁글구', '고저', '립하', '피터', '상대원동', '경상', '주년', '다라', '디지털타임스', '살뻔', '년차', '맏으', '예뻣', '사만원', '날밤', '원평', '고춧가루', '강변', '어찌', '개꼴', '헬로', '개중', '거중', '광명', '입홉용', '먹음었다', '고운동', '만하', '한평생', '들었습돠', '거사', '잌체', '이민수', '가젹', '희구', '장복', '홈피', '꼬박', '햇던', '뒤편', '보임', '다음주', '종국', '동이', '거지', '실지로', '며폐', '주엇습니', '문구로', '처음', '해댕', '강의', '요리조리', '셧길레', '알구', '맹비', '지히', '마르', '어디', '고마움', '찌리', '먼저', '길동', '저기', '힙니', '도통', '능이', '빠릴', '고니', '춘추전국시대', '헝어엉', '라른것', '딸라', '들으셧는', '관악구', '조이', '갑지', '돈덜', '갈수', '일임', '온통', '남양주시', '튀어', '가경', '바나나킥', '다지', '이즈', '사드', '프사', '낭낭', '도랑', '서서히', '리즌', '너희', '무슨', '무턱', '나왓습니', '찌꺼기', '흥건해', '수단', '기망행위', '로스트', '예나', '유독', '구장', '걸렷', '꼽아놓', '신방화역', '제발', '잃으킨다', '배영환', '그대', '누루던', '덕소', '역곡동', '온갖', '저가살떄', '다합', '이나고', '블락', '특화된', '모래알', '핑곕니', '마족', '천안시', '빨간색', '다주', '마루', '공팟', '후반', '난건', '근처', '툴툴거렸', '배행', '예상외', '울산', '턱턱', '훗날', '어째서', '건데제', '번지', '겨울', '이름', '쭐어', '런가', '놈임', '흰색', '값임', '여의도', '한명인', '이경우', '년안', '스웨', '성주읍', '메리엇', '위엣분', '뉴수', '도제', '손안', '개됫', '슬쩍슬쩍', '미수', '행궈', '바떼', '호호', '이채림', '다대고', '노보', '라다', '체킁후', '진짜루', '불때', '짝뚱', '것과', '확락환', '영위', '딱지', '깉습', '성황리', '거격', '려고', '꼿히', '개념', '하셧는데', '나물', '냉정역', '이드', '뎁니', '보렐리', '가장', '무스', '모회사', '기피', '며연', '해악', '벌이', '하젝', '님게', '덜렁', '저리', '이쯤', '이제', '낫다', '프러포즈', '성기구', '퍼뜨립니', '천곡동', '사시', '더헤', '레보', '가야읍', '국뽕', '구들', '렵니', '죙일', '되닌', '모른체', '날짜', '마셧', '무수', '물기', '고고', '동아닷컴', '짱짱', '부어', '보셧', '닝안', '까오', '방시', '공항', '좋아졋', '만발', '얺으실것', '또술', '칼칼', '니지', '톨루이', '지극', '땜빵', '글세요', '프라', '전남', '해해', '두시', '순천시', '각지', '타닥거릴', '제껀', '최대한', '라만', '상도', '완젼', '오브', '찻으시는것', '힌색', '천만', '디오', '컴퓨터게임', '흠칫', '달리', '나음', '나프', '간주', '캐캐', '독착', '때무넹', '보심', '황후', '낭설', '문자', '제로', '럼체택', '괌쏴합니', '해대', '단위', '구성동', '거기', '미르', '하얀색', '세배', '유일하', '낼때', '일로', '나왓다', '성북구', '그형껀', '꿀꺽', '게이', 'ㅂ이', '영통', '지고', '하룻밤', '만인', '맼모드', '가쁘', '천송동', '꾸벅', '제방', '숫불', '걸알', '쉽사리', '다시피', '매년', '모두', '론거', '분홍색', '도아', '여수시', '안눌려', '리나', '비레', '평일', '제하', '일도', '제대', '때전', '이요', '고해', '치면', '루시', '답장', '전혀없어', '당일', '엄청쌥니', '곧바로', '도충', '덕양구', '성의껏', '라면', '몸에늨', '싸이', '하부', '남구', '흐릿흐릿하', '마흐무드', '모르겄', '쇼트나', '성도', '당빠', '전날', '질겅질겅씹어', '옴제', '감고', '붇거', '생격', '올해', '훨얼', '김제', '줄알', '조물', '꼬애과리', '조은준', '지만원', '가빠오던', '제각기', '주더', '퍼온', '계속', '남양주', '민가', '위져', '무향쪽', '일부러', '산다', '되레', '리론', '신종은', '다현', '독창', '소홀히', '여부', '벨린다', '뜨렷', '번갈아', '요사', '팔구', '장동민', '는가', '용산', '득달', '태백', '사활', '마을', '톱니', '괞찮', '메가박스', '헤어지자', '문해', '빠샤', '키때매', '라마', '제시', '디즈니랜드', '새삼', '횡성군', '척척', '마주', '저쪽', '갱목', '꼽으', '빼주', '듭니다', '끼리', '담결', '줄줄이', '개이득', '우연찮', '구구', '뭋치', '더보', '거슬러', '영종도', '거증', '아예', '송해', '똥차', '년대', '고추가', '질도', '십조원', '가라', '떙기면', ' 되엇어', '듬니', '태인', '로움', '이나', '장액', '커서', '날로', '마춤', '헹군', '구함', '광진구', '덧글', '커플', '주간동아', '도임', '힌번', '부킹', '존나', '남도', '피고', '현재', '종도', '쪼가리', '왓으', '동래', '번가', '결코', '꼽을수', '중원', '헤헤', '나왓', '갑필때', '엠버', '분당', '성주군', '미안', '픽사', '디어', '번앤', '다집', '맥킷', '오후', '란셋', '홀릭이', '글쓴님', '이걸루', '전남자친구', '느기시', '나가시', '백년', '남아', '리나로', '만들죵', '현격히', '온건', '리하', '견줘', '조선', '눌림', '남녀', '라미아', '계월', '전부', '이제껏', '바텀', '폇습니', '디알', '박상준', '페이', '상대로', '피구', '멀리', '퍼플', '값외', '그남자', '전혀', '정보라', '꿀맛꿀맛', '맙니', '답습', '무르익자', '우선', '미지', '티브이데일리', '대연', '줄곧', '쪼금', '홍관', '차마', '송정역', '뭐더', '다스', '젹셔져', '강구', '미사', '던데', '어딘가', '매우', '깨름직', '안함', '눈차쿠', '배겨', '순천', '무연', '완전', '가판', '후크', '님같', '끄나풀', '더크', '많은량', '뽐빵', '갑변', '크로마', '리리', '다엽', '있습니딘', '꼼새', '니미', '솜등', '여름', '꼽거', '적격입니', '됫으', '서프라이즈', '됫었습니', '이튼', '무자', '네덜란드', '갈고', '도리', '물건', '사실빠', '덜컥', '취합해', '태의', '가가', '곳좀', '희망이', '찜함', '눈매', '온곳', '정말', '금영', '자뀨', ' 반드시', '희망', '곳었으', '겟네', '문외한', '드른건', '말임', '준후', '하셧자나', '한강로', '나중', '로시', '통계', '달좀', '박대', '백만원', '주로', '제실', '동호', '달짝찌', '래야', '각각', '느지', '어쨌던', '진짜진짜', '보라색', '차서', '시망', '비주', '아마', '물체', '리빌드해', '만금', '감사용', '흥건', '삼일만', '돈좀', '드믈지', '이왕이면', '여하튼', '과거', '맛으', '디다', '돕기', '만용', '중국어', '대가리', '이레', '작성자', '몇칭', '일종', '무언가', '리터', '함안', '질리', '새로이', '오산시', '대서', '신창호', '모이', '남성', '거의', '것임', '내달', '디스', '년후', '덜드나', '겸사겸사', '전화번호', '부위', '미카엘', '다음', '대거', '부러', '떨어트려', '택할', '리얼', '뚜둥', '동대문', '격입니', '하이', '어제', '길다랗', '그득', '개비', '가기', '무르', '도편', '강츄', '하려닌깐', '여느', '안', '한국말', '메일', '통방', '늅늅', '셧던', '프라스', '앞서', '스냅', '곧장', '맛족', '마치', '아므튼', '디프', '나둿습니', '때목', '이번', '얄라', '스캣', '라이머', '흡입압', '내용', '닝닝해', '꼽으라', '루고', '만듬', '임형철', '당분간', '회계사', '가령', '혼환', '일일', '덴마크', '물이', '대도', '공개', '내리락', '하니', '다사', '십여', '주무', '강연', '물량들', '수나', '네시', '넣기', '하셧네', '미화', '고인', '그동안', '헬스', '반드시', '달반', '건너', '싶어할껀데', '플로', '제주도', '밍숭맹숭', '오늘날', '로만', '필때', '전월', '조차', '야동', '는데', '김대진', '다모', '꼽는', '누군가', '드럿', '그대로', '피우지', '디랙', '몰이', '투성이', '단일', '으루', '주요', '생으로', '노인성', '망우', '그것땨뮨', '주필', '순이', '우측', '구점', '파란', '주파', '글쓴이님', '무현', '개개인', '가로', '낮낮', '마다', '해봣', '만난', '개봉', '게뿔', '더비', '하자', '곳곳', '거박', '담변좀', '사람', '성상', '중구', '도소', '구분', '살때', '먹음', '뚝딱', '피나', '다짜고짜', '감배', '착케착케', '전선꼽는쪽', '달이', '타나', '다모아', '세로', '머니투데이', '장천', '박용', '매하', '쉽지요', '오리', '초초해', '나왓어', '남천동', '할떄', '와중', '물좀', '위쪽', '어쨋', '모씨', '힘껏', '마자', '모하', '담대', '것습니', '쫨씩', '부숴진', '무려', '맥심', '진선미', '마당', '귀챦기', '두고두고', '여주', '여기', '거리', '되닌깐', '홍종필', '솜일', '거여', '이와지사', '걸또', '함페수', '네바다', '솔솔', '여러', '뛸껍니', '도참', '징둥', '답니', '조만간', '단연', '달전', '름선', '충청', '꼬매버리', '영통구', '널렷습', '광양', '난또', '별로', '단명', '반년', '삿을때', '년전', '펴보닌깐', '폇던', '햇으', '똬악나와', '보아', '진작', '쓰닥', '시라', '산이', '보이지', '비바', '쭐이시다', '하란', '천송', '틈틈이', '평촌역', '각기', '한순간', '제일', '얼마나', '구의동', '불과', '살떄', '구역', '번은', '정신차려라', '궁굼', '십원', '서현동', '고랑', '투데이', '디메', '헤봣', '타지', '원미동', '뽐핑하', '고작', '니코니코니', '로더', '코로', '갈기', '만이구', '제이', '어프', '천지', '요즘', '를좀', '요런', '고요', '감가상각', '통한', '개명', '는걸', '통상', '뜸금', '이투데이', '커녕', '동일시', '루트', '얼마', '천천이', '신지', '성남시', '송도', '어자보', '순다', '자꾸나', '봉로', '맛표', '구포동', '어치', '피더', '통계청', '남현주', '부천역', '그량', '등꼴휘', '날대', '진주시', '리가', '간간이', '망정', '믹서', '장날', '후덜더럳류', '나하', '나은', '엇을', '옹호', '라이', '만약', '하여튼', '리크', '소기', '말초', '그간', '엊그제', '김재호', '뎃글', '건수', '뀌진', '이츰', '되겠스비낟', '게이지', '지원이', '대주', '클렌징폼과같', '따위', '허것입니', '올바로', '롯켓하', '개월차', '도록', '지난해', '쳐앰플', '니코친', '매수', '쥑입니', '에두', '성남', '니요', '노소', '옷등', '자지인', '물질', '빨때', '한시', '부탑', '헹구', '미처', '현제', '미려', '벙벙', '보지', '깨끛', '누구', '강서', '어디껀', '여차하면', '리해', '내세', '지네', '아시', '흐트러', '주지혜', '미원', '부평구', '민망', '시어', '모이라', '지난', '대리', '취했다', '당장', '콜콜', '라거', '때발', '드네', '줄넘기', '땜시', '매몰비용', '론자', '쏱아져', '원도', '레알', '강아지', '개월전', '괜찬은거', '장착킷', '각종', '동구', '드료', '감등', '하죵', '김종오', '금년', '스포츠한국', '한림대', '간체', '자하', '걸루', '김재열', '봉래동', '한편', '보라', '살이', '참으로', '청주시', '밑면', '까마귀', '비공개', '십년', '피셧던', '첫째', '금은', '됫습니', '좀처럼', '피온', '오른쪽', '꼽았다', '강남', '눈기', '오전', '충북도', '더러', '겟거', '눌리시', '했엇습니', '꼽아놔', '망성', '로데오', '간단명료', '여태껏', '최우', '스포츠조선', '해운', '실었다', '힐끔', '색갈', '엠넷', '안팎', '택했고', '먆게', '무르익', '도림', '꽐라되', '머델', '약간', '톨루엔', '물컵', '기종은', '공주대', '김은지', '제이지', '니아', '고양', '나리', '마곡', '일본어', '가미', '마지막', '네임택', '라합', '고여', '과외', '나무', '무화량많', '턴데', '마나', '스랑', '대게', '널살', '돌기', '눈챠크', '김해', ' 공덕동', '허다해', '대전광역시', '꼭체택', '테움', '향후', '매부', '고거', '부터', '자옆', '때메', '구찮기', '한겨례', '다운동', '찬물', '되지않읆까', '타롯', '덩어리', '바비', '체택', '컬투', '김진호', '충천', '칭할정도', '핣으', '어가', '들쭉', '쭈울', '로우', '눈탱', '갈리', '환풍기', '고루', '컴퓨터', '슬림해', '나노', '덜커덕', '충분', '다국적', '최악', '박아', '페텍', '떗을떄', '최선', '다그', '햤으', '오지랖', '꿀떡', '역시', '월트디즈니', '일차', '개국', '활짝', '한걸음씩', '로거', '누운', '맥스', '리처드', '너렷습', '자고', '우후', '년도', '대별', '한결', '수록', '다작', '타가', '동안구', '물어', '피기', '잠자리', '지드', '들것', '때감', '면목역', '괞찮겠', '압니다', '대개', '민후', '버졋', '가오', '측만된', '어그', '게목', '팁쪽', '러시아', '이천', '휴게소', '거꾸로', '둔산동', '름좀', '살걸', '등등', '자츰자츰', '펴봣는대', '키기', '뚝뚝', '편임', '해당', '기웃', '뻐끔', '그냥', '한마디', '연합뉴스', '들불', '나멍', '최대로', '수원시', '장아찌', '단언', '강릉시', '가을', '나바호', '에스', '평생', '바야흐로', '셧으', '가트', '오늘밤', '굼금합니', '계란', '쇼트나서', '간단', '쟁이들', '응근히', '조사해', '달랑', '건보', '아아', '시나', '므화기', '부득이', '항간', '연일', '으름', '뻔뻔', '크기', '맥시', '제주', '겔럭시', '일요일', '사두', '있', '에전', '종이니', '덕분', '저딴', '찰나', '금세', '많쵸머', '며느리', '됫다', '하하', '햇습니', '헬스베', '무엇', '누님', '안나', '서사', '이로', '잠시', '다빨렸는데', '퍼뜨리', '홍문기', '구진', '나븐', '되셧', '탑캡쪽', '햇놧', '구가', '어란', '리브가', '메비우스', '은평', '여개', '일단', '루드', '무가', '흐물흐물', '텁텁한건', '블러', '렬구', '재팬', '때마침', '리빌드란', '결국', '가끔', '쫘악', '정이', '베이', '다미', '란건', '미리', '레드', '라반', '그네', '본격', '마구', '만도', '글썽', '짝짝', '쪼만', '극도', '네임카드', '저만큼', '나라', '스스로', '남자에게', '문의', '이조', '이형태', '윤희', '너엇다', '한낮', '대체로', '하라', '안이', '몫일', '김지훈', '뽐뿌', '자마자', '흐흐흐내', '할로', '웹툰', '트렸', '면대', '연희', '오락', '로나', '하셧나', '오닐', '소나', '나좀', '괜찬아', '니나', '말씀', '무니', '안눌럿는데', '더즈', '동아', '람보', '한국어', '화이트', '굴뚝', ' 마야인', '쭈쭈바', '특별', '달구', '김지연', '랜드', '덕소역', '하시느', '리도', '기두', '학년', '마저', '닉솜', '페페', '돠었습니', '칭하', '밤새', '타이어', '지경', '뭉뚱그려', '강원도', '잘해야', ' 신현재', '쉬이', '향남', '마련', '고개', '의정부', '을해', '지적도', '이사야', '오토바이퀵', '흐지부지', '브라운', '엠펭', '탁탁', '행여', '그페헤', '해약', '일각', '바탕', '직접', '삼평동', '이탭', '끄덕', '쿠키뉴스', '은줄', '중원구', '개별', '리지', '도하', '잔량들', '올린다', '흐흣', '명동', '나라고', '및액', '뚤리', '로라', '등잉', '매니', '보신', '한쪽', '래서', '구한', '정작', '막피', '슬슬', '듯해', '누차', '돼지고기', '나시', '자시', '뉴데일리', '얼마간', '가득', '드드드', '뿌둥', '제니', '라서', '리다', '메리', '레스', '잘잘', '질질', '가다가', '폇어', '달액', '눈빛', '이아', '여전', '감동', '먹듯', '경기도', '즐비', '긱베', '원랜', '떄에따', '백유진', '솜외', '서여', '나고지', '춘천', '수십', '광천수', '좋쿠', '로그인', '만요', '자주', '금사항', '마음대로', '쪼그마', '그닥', '대강', '행낭', '한겨레', '다년', '선뜻', '것좀', '로데오거리', '잡기', '흐흑', '궁굼해하셧던', '관련성', '볼땐', '납짝', '그거', '했을껄', '그때', '거니', '처넌', '액생', '그먕', '직빵', '치가', '내일', '뉴시스', '들통', '던람', '지자', '쩐다', '네가지', '쾌쾌', '새해', '덜피', '글쓴', '매스', '에리', '가량', '루프', '지다', '껀데', '시더', '란걸', '나날', '까닭', '정도', '덩이', '마켓', '다크', '시지', '한가지', '가면', '되욤', '통해', '도부', '따름', '점차', '수년', '마곡역', '고려대학교', '츄잉', '최저', '무것', '빅뉴스', '우러나', '구우', '다정', '김동엽', '혹시', '조마조마', '전화', '여러가지', '중화산동', '리판', '그로', '누굴', '입항', '때제', '삼일로', '근래', '외진', '과천', '피삼', '부디', '네이버', '안산', '가비', '뽐삥하', '훗딱', '강하', '꼬아두', '간다라', '김경훈', '진절머리', '토요일', '더프', '구리', '놀란', '은박', '소관', '갈수록', '필히', '효울', '나타', '개똥', '성은', '듬뿍', '된거', '놀러와', '도타', '아ㅖ', '창전동', '대일', '천차만별', '은색', '롯켓엔', '송료', '모시', '나타남', '흠뻑', '가배', '주엽역', '공항동', '습니', '청주', '화성시', '구의', '가운데', '그래서', '불어', '한밤중', '서울시', '홍준기', '연산동', '거가', '뽐삥', '대야', '심지어', '최진수', '하등', '이동엽', '박종윤', '멋쩍', '원레', '거르', '영영', '탭캡', '휘달렷', '빠꾸어', '달인', '거즘', '단배', '더베', '경주시', '순차적인', '그나', '제각각', '커플사진', '수일', '아니냔', '혹자', '푸르락', '잡고', '이기', '오빠', '펴왓는데', '장천동', '뭉겔', '개뿔', '애시', '자칫', '건호', '곳임', '빠저', '요건', '절대로', '있죵', '리기', '모라', '내야', '조그만', '꾸리', '훨신', '기일', '친것', '번꼴', '일부', '크로스', '웃기', '따따', '해봣자', '베이비', '하루', '퐁당', '혹여', '말귀을', '이상만', '간혹', '번번이', '구미시', '신가', '며칠', '일어나지', '때론', '최상', '쩌구저쩌', '눈쌀', '오디오', '사대', '섵택해', '고항', '게속', '기타', '무화량등', '속으', '괞히', '덩달아', '일기', '어쨋든', '문득', '난적', '다누', '여태', '무크', '여제', '뭉치량', '체택좀', '하연', '뚜뚜뚜뚱', '어쩌', '졸졸', '그레', '폇는데돈', '소공동', '나율수', '돌배효', '당제', '잠기', '순간', '맛셋트', '것이므', '제위', '핫스판', '말입', '서로', '성부', '못썻구', '엣지', '가사', '도데', '삿는데', '당초', '먹방', '별거', '시킴', '한뎁', '을지로', '도저히', '돌팔매', '고양시', '충청북도', '만타', '매트', '개체', '건너편', '가지각색', '빨렸', '불똥', '드내', '오거리', '무화량때매', '이데일리', '이해해', '오직', '마냥', '덜덜', '언넝', '에이', '블랙', '만가', '강당', '그야', '때로는', '입호읍']

diff_data = list(set(documents1).difference(stop_list))
documents2 = [diff_data]
dictionary = gensim.corpora.Dictionary(documents2)

print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가격': 0, '가능': 1, '가능성': 2, '가열': 3, '가이드라인': 4, '가족': 5, '가향': 6, '간과': 7, '간접흡연': 8, '갈등': 9, '감각': 10, '감기': 11, '감사원': 12, '감소': 13, '강력': 14, '강요': 15, '강제': 16, '강조': 17, '강화': 18, '개발': 19, '개선': 20, '개요': 21, '개인정보': 22, '개인정보보호법': 23, '개정': 24, '개정안': 25, '갤러리': 26, '거부': 27, '거주': 28, '거짓': 29, '걱정': 30, '건강': 31, '건물': 32, '건축': 33, '검사': 34, '검색': 35, '검시': 36, '검진': 37, '검찰': 38, '검토': 39, '게시': 40, '게임': 41, '게재': 42, '격려': 43, '격화되': 44, '결과': 45, '결단': 46, '결론': 47, '결심': 48, '결정': 49, '결핵': 50, '결혼': 51, '결혼식': 52, '경계': 53, '경고': 54, '경로': 55, '경악': 56, '경쟁': 57, '경쟁력': 58, '경험': 59, '계기': 60, '계산': 61, '계층': 62, '계획': 63, '고객': 64, '고글': 65, '고등학교': 66, '고등학생': 67, '고민': 68, '고발': 69, '고백': 70, '고생': 71, '고시': 72, '고지서': 73, '고지용': 74, '고통': 75, '고형': 76, '골자': 77, '골초': 78, '공간': 79, '공감': 80, '공격': 81, '공고': 82, '공공': 83, '공공기관': 84, '공공시설': 85, '공공장소': 86, '공기': 87, '공단': 88, '공동': 89, '공론': 90, '공률': 91, '공무원': 92, '공사': 93, '공세': 94, '공수': 95, '공원': 96, '공유': 9

In [37]:
corpus_path = '전자담배 금연_naver_news_1000.json'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [38]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[]
[]
[]
[]
[(185, 1)]
[]
[]
[(801, 1), (887, 1)]
[]
[]
[]
[]
[]
[]
[(1206, 1), (1468, 1)]
[]
[]
[]
[(444, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(732, 1)]
[]
[]
[]
[(51, 1)]
[]
[]
[(731, 1)]
[]
[]
[]
[]
[]
[]
[(109, 1), (1412, 1), (1482, 2)]
[]
[]
[]
[(1073, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(485, 1), (552, 1), (660, 1), (675, 1), (1361, 1), (1484, 1)]
[]
[]
[]
[(552, 1)]
[]
[]
[(83, 1), (707, 1), (1014, 1), (1482, 1)]
[]
[]
[]
[]
[]
[]
[(485, 1), (660, 1), (675, 1), (1361, 1), (1484, 1)]
[]
[]
[]
[(552, 1)]
[]
[]
[(485, 1), (660, 1), (675, 1), (1361, 1), (1484, 1)]
[]
[]
[]
[(855, 1)]
[]
[]
[(41, 1), (459, 1), (952, 1), (1460, 1), (1482, 2)]
[]
[]
[]
[(1482, 1)]
[]
[]
[]
[]
[]
[]
[(1482, 1)]
[]
[]
[]
[]
[]
[]
[(51, 1)]
[]
[]
[(819, 1), (1369, 1)]
[]
[]
[]
[(185, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(731, 1)]
[]
[]
[(51, 1)]
[]
[]
[]
[(51, 1), (739, 1)]
[]
[]
[(459, 1), (1460, 1)]
[]
[]
[]
[]
[]
[]
[(459, 1)]
[]
[]
[]
[(1206, 1)]
[]
[]
[]
[]
[]
[]
[(46, 1)]
[]
[]
[]
[]
[]
[]
[(818

[]
[(901, 1)]
[]
[]
[]
[]
[]
[]
[(48, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(247, 1), (501, 1), (914, 1)]
[]
[]
[]
[(63, 1)]
[]
[]
[]
[]
[]
[]
[(495, 1)]
[]
[]
[(184, 1), (1052, 1)]
[]
[]
[]
[(1102, 1), (1337, 1)]
[]
[]
[]
[]
[]
[]
[(732, 1)]
[]
[]
[]
[]
[]
[]
[(541, 1)]
[]
[]
[(907, 1)]
[]
[]
[]
[(1228, 1)]
[]
[]
[(451, 1)]
[]
[]
[]
[(893, 1)]
[]
[]
[]
[]
[]
[]
[(732, 1)]
[]
[]
[(732, 1)]
[]
[]
[]
[]
[]
[]
[(51, 1)]
[]
[]
[]
[(541, 1)]
[]
[]
[]
[]
[]
[]
[(967, 1)]
[]
[]
[]
[]
[]
[]
[(63, 1), (541, 1)]
[]
[]
[(732, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(1121, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(967, 1), (1239, 1)]
[]
[]
[]
[]
[]
[]
[(184, 1), (589, 1), (1052, 1), (1386, 1)]
[]
[]
[]
[(225, 1)]
[]
[]
[(525, 1)]
[]
[]
[]
[(930, 1)]
[]
[]
[(184, 1), (1052, 1)]
[]
[]
[]
[(1102, 1), (1337, 1)]
[]
[]
[(184, 1), (817, 1), (874, 1), (1269, 1)]
[]
[]
[]
[(121, 1), (1337, 1)]
[]
[]
[(184, 1), (305, 1), (817, 1), (874, 1), (1269, 1)]
[]
[]
[]
[(1102, 1)]


[]
[]
[]
[]
[(229, 1), (463, 1), (791, 1)]
[]
[]
[]
[(247, 1), (1474, 1)]
[]
[]
[(162, 1)]
[]
[]
[]
[(31, 1), (1206, 1)]
[]
[]
[(906, 1)]
[]
[]
[]
[(247, 1)]
[]
[]
[(247, 1), (588, 1), (960, 1)]
[]
[]
[]
[(247, 1)]
[]
[]
[(182, 1), (184, 1), (373, 1)]
[]
[]
[]
[(376, 1)]
[]
[]
[(274, 1), (1368, 1), (1482, 1)]
[]
[]
[]
[(1216, 1)]
[]
[]
[(1418, 1)]
[]
[]
[]
[(69, 1)]
[]
[]
[(381, 1), (979, 1)]
[]
[]
[]
[(38, 1), (1377, 1)]
[]
[]
[(184, 1), (567, 1)]
[]
[]
[]
[]
[]
[]
[(851, 1)]
[]
[]
[]
[(38, 1), (121, 1), (1377, 1)]
[]
[]
[(158, 1), (947, 1)]
[]
[]
[]
[]
[]
[]
[(154, 1), (318, 1), (1395, 1), (1487, 1)]
[]
[]
[]
[(574, 1)]
[]
[]
[(32, 1), (198, 1)]
[]
[]
[]
[(467, 1)]
[]
[]
[(924, 1), (1078, 1), (1420, 1)]
[]
[]
[]
[(1420, 1)]
[]
[]
[(158, 1), (265, 1), (943, 1)]
[]
[]
[]
[(1327, 1)]
[]
[]
[(670, 1)]
[]
[]
[]
[]
[]
[]
[(135, 1), (714, 1), (784, 1)]
[]
[]
[]
[(1206, 1)]
[]
[]
[(135, 1), (278, 1), (454, 1), (1206, 1)]
[]
[]
[]
[]
[]
[]
[(943, 1)]
[]
[]
[]
[(516, 1)]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[(247, 1), (678, 1), (1136, 1)]
[]
[]
[]
[]
[]
[]
[(247, 1), (1078, 1), (1136, 1)]
[]
[]
[]
[]
[]
[]
[(247, 1), (1078, 1), (1136, 1)]
[]
[]
[]
[(1280, 1)]
[]
[]
[(247, 1), (1078, 1), (1136, 1)]
[]
[]
[]
[(1280, 1)]
[]
[]
[(18, 1), (642, 1), (1027, 1), (1334, 1)]
[]
[]
[]
[(121, 1), (1062, 1)]
[]
[]
[(537, 1)]
[]
[]
[]
[(947, 1)]
[]
[]
[]
[]
[]
[]
[(1102, 1), (1482, 1)]
[]
[]
[(31, 1), (37, 1), (50, 1), (820, 1), (1080, 1)]
[]
[]
[]
[(31, 1), (257, 1), (373, 1)]
[]
[]
[(471, 1), (805, 1)]
[]
[]
[]
[(109, 1), (1027, 1), (1102, 1)]
[]
[]
[(247, 1), (707, 1), (1014, 1), (1386, 1)]
[]
[]
[]
[(109, 1)]
[]
[]
[(31, 1)]
[]
[]
[]
[]
[]
[]
[(31, 1)]
[]
[]
[]
[(257, 1), (373, 1)]
[]
[]
[(537, 1), (678, 1)]
[]
[]
[]
[]
[]
[]
[(1460, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(45, 1), (732, 1)]
[]
[]
[(441, 1), (471, 1), (594, 1), (1253, 1)]
[]
[]
[]
[(227, 1)]
[]
[]
[]
[]
[]
[]
[(732, 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(965, 1)]
[]
[]
[(537, 1), (678, 1), (1464, 1)]
[]
[]
[]
[]
[]
[]
[]


In [39]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=1488, num_topics=10, decay=0.5, chunksize=2000)


In [40]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [41]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 1488)
1.0


In [42]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

In [43]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)

In [ ]:
pyLDAvis.show(prepared_data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [23/May/2019 12:18:34] "GET / HTTP/1.1" 200 -
